In [2]:
import json
import glob, os
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import xmltodict
import pickle
ps = PorterStemmer()
lm = WordNetLemmatizer()

dbfile = open('globalDump', 'rb')      
db = pickle.load(dbfile)
dout = {}
for keys in db:
    dout[keys] = db[keys]

In [6]:
import xml.etree.ElementTree as ET


def getLevel( root ):
    if len( root ) == 0:
        return set()
    else:
        ret = set()
        for child in root:
            if 'certainty' in child.attrib and int(child.attrib['certainty']) >= 0:
                ret.add( child.attrib['value'].lower().strip() )
                if 'cui' in child.attrib:
                    for ele in child.attrib['cui'].split(";"):
                        ret.add( ele.lower().strip() )
            ans = getLevel( child )
            ret = ret.union( ans )
        return ret



Patients = dout['Patients']

for i in range(len(Patients)):
    base_path = ".'/"
    pat = Patients[i]
    pat.documentContentList = []
    if not os.path.isdir( base_path ):
        continue
        
    for item in pat.documents:
        try:
            tree = ET.parse( base_path + item['DocName'] + ".xml" )
            root = tree.getroot()
            ans = getLevel( root )
            pat.documentContentList.append( ans )
        except:
            continue
Patients[0].documentContentList

[{'abscess',
  'airway',
  'antibiotics',
  'asthma',
  'bilateral',
  'bp',
  'c0000833',
  'c0001688',
  'c0003232',
  'c0004096',
  'c0005823',
  'c0005824',
  'c0008196',
  'c0009421',
  'c0009450',
  'c0011344',
  'c0012634',
  'c0013227',
  'c0013621',
  'c0013622',
  'c0013658',
  'c0017714',
  'c0020538',
  'c0020740',
  'c0023376',
  'c0023660',
  'c0023879',
  'c0024320',
  'c0030193',
  'c0030840',
  'c0030842',
  'c0032821',
  'c0037623',
  'c0039401',
  'c0040426',
  'c0040460',
  'c0041755',
  'c0042769',
  'c0055361',
  'c0057191',
  'c0124647',
  'c0162800',
  'c0175659',
  'c0178987',
  'c0184617',
  'c0202194',
  'c0202404',
  'c0205090',
  'c0205091',
  'c0220836',
  'c0220892',
  'c0222045',
  'c0226896',
  'c0226984',
  'c0230028',
  'c0234226',
  'c0237849',
  'c0238767',
  'c0242724',
  'c0242936',
  'c0282687',
  'c0304475',
  'c0349674',
  'c0352155',
  'c0424927',
  'c0439092',
  'c0439824',
  'c0442027',
  'c0458827',
  'c0475211',
  'c0518988',
  'c0546869',

In [1]:
import pickle
from PatientRecord import *
dbf = open("mGD","rb")
db = pickle.load(dbf)
dbf.close()

In [2]:
Patients = db['Patients']
corpus = [['']]*( len(Patients) )
for i in range(len(Patients)):
    if len( Patients[i].documentContentList ) > 0:
        ss = Patients[i].documentContentList[0]
        for j in range(1,len(Patients[i].documentContentList)):
            for ele in Patients[i].documentContentList[j]:
                ss.add( ele )
        corpus[i] = list( ss )
corpus[0]

['bilateral',
 'c0238767',
 'mouth',
 'c1555764',
 'ibuprofen',
 'c0234422',
 'alert',
 'lower jaw',
 'c1281562',
 'ibuprofen 800 mg',
 'c0239110',
 'pulse',
 'c0042769',
 'c0227093',
 'c2239132',
 'c0226896',
 'c1705313',
 'c0013604',
 'c0034625',
 'c0012634',
 'temperature',
 'c1456781',
 'c0324740',
 'c0202404',
 'c0228062',
 's00004748',
 'c0022363',
 'mild_sep_swelling',
 'potassium',
 'jaw area',
 'c0027960',
 'c0020538',
 'c1535146',
 'c0175659',
 'c0031354',
 'c0001688',
 'jaw',
 'is tooth',
 'c0024320',
 'gingival swelling',
 '2: tooth',
 'c1424810',
 'c1552823',
 'c1278910',
 'kristin',
 'c3273340',
 'c0222045',
 'c0003232',
 'awake',
 'c0000833',
 'abscess',
 'c0017714',
 'hypertension',
 'c1947916',
 'c2210645',
 'c1522412',
 'penicillin',
 'c1527342',
 'lower left molar',
 'c2984299',
 'c1522187',
 'swelling',
 'c0304475',
 'c3536777',
 'gingival',
 'c0439092',
 'c0003873',
 'c1557397',
 'c1552822',
 'lower',
 'verbal',
 'c0031809',
 'c0439824',
 'c3146279',
 'c0718338',
 

In [3]:
import numpy as np
from gensim import corpora
from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity
               
sims = {'files': {}}
dictionary = corpora.Dictionary(corpus)
corpus_gensim = [dictionary.doc2bow(doc) for doc in corpus]
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]

In [4]:
import pandas as pd
dat2 = pd.read_csv("./RG2/Case_Documents.csv")
for index,row in dat2.iterrows():
    corpus[ row['CaseId'] ].append( row['DocType'].lower().strip() )
corpus[5795]

['',
 'outbound phys note',
 'edphynotes',
 'outbound phys note',
 'edphynotes',
 'nurse',
 'ednurnotes',
 'outbound phys note',
 'edphynotes',
 'outbound phys note',
 'edphynotes',
 'nurse',
 'ednurnotes',
 'nurse',
 'ednurnotes',
 'edphynotes',
 'outbound phys note',
 'nurse',
 'ednurnotes',
 'ednurnotes',
 'addendum',
 'outbound phys note',
 'edphynotes',
 'pd.obs.h&p',
 'nurse',
 'ednurnotes',
 'perc drain',
 'pd.obday1',
 'obprog',
 'nurse',
 'ednurnotes',
 'outbound phys note',
 'edphynotes',
 'transvag',
 'gyn.con',
 'addendum',
 'addendum',
 'nurse',
 'ednurnotes',
 'edphynotes',
 'outbound phys note',
 'abd/pel w',
 'nurse',
 'ednurnotes',
 'outbound phys note',
 'edphynotes',
 'nurse',
 'ednurnotes',
 'outbound phys note',
 'edphynotes',
 'abd/pelw/o',
 'nurse',
 'ednurnotes',
 'outbound phys note',
 'edphynotes',
 'consur',
 'amb h&p',
 'postproc',
 'mqoprep',
 'nbn.del',
 'nb admit',
 'nbnattprog',
 'dcsum',
 'dcsum',
 'nurse',
 'ednurnotes',
 'outbound phys note',
 'edphyn

In [5]:
data = pd.read_csv('Demo_processed.csv')
from sklearn import preprocessing
import pandas as pd
cols = list(data.columns.values)[1:3]
x = data.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled)
data = data.drop( columns=[0] )

In [6]:
for i in range(len(cols)):
    dictionary.add_documents([ [cols[i]] ])
    curr_id = dictionary.doc2idx([ cols[i] ])[0]
    print(i)
    for j in range( len(Patients) ):
        corpus_tfidf[j].append( ( curr_id , data.at[ j , i+1 ] ) )

0
1


In [7]:
lsi = LsiModel(corpus_tfidf, id2word=dictionary)
lsi_index = MatrixSimilarity(lsi[corpus_tfidf])
sims['files']['LSI'] = np.array([lsi_index[lsi[corpus_tfidf[i]]] for i in range(len(corpus))])

In [8]:
sims

{'files': {'LSI': array([[1.        , 0.17196947, 0.01228897, ..., 0.0197802 , 0.04383558,
          0.04696609],
         [0.17196947, 1.        , 0.04258695, ..., 0.00388564, 0.0984126 ,
          0.00981943],
         [0.01228897, 0.04258696, 1.        , ..., 0.12093639, 0.08612388,
          0.06313406],
         ...,
         [0.0197802 , 0.00388564, 0.1209364 , ..., 1.        , 0.05085507,
          0.04618831],
         [0.04383558, 0.09841265, 0.08612388, ..., 0.05085509, 1.        ,
          0.11396601],
         [0.04696609, 0.00981942, 0.06313406, ..., 0.04618831, 0.113966  ,
          1.        ]], dtype=float32)}}

In [9]:
ind=np.unravel_index(np.argmax(sims['files']['LSI'],axis=None), sims['files']['LSI'].shape)

In [10]:
k = sims['files']['LSI']

In [11]:
k[6153][857]

0.8149763

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(k)
k2 = scaler.transform( k )
k2

array([[1.        , 0.20669086, 0.08456239, ..., 0.0769209 , 0.08419853,
        0.12754476],
       [0.21549496, 1.        , 0.11264339, ..., 0.0619529 , 0.13647167,
        0.09353883],
       [0.06420806, 0.08273369, 1.        , ..., 0.17218032, 0.1247017 ,
        0.14234573],
       ...,
       [0.07130551, 0.04565522, 0.1852598 , ..., 1.        , 0.0909217 ,
        0.12683274],
       [0.09409642, 0.13621847, 0.1529946 , ..., 0.10618433, 0.99999994,
        0.18887985],
       [0.09706238, 0.05134018, 0.13168702, ..., 0.10178959, 0.1513685 ,
        1.        ]], dtype=float32)

In [13]:
k2[6153][857]

0.82009107

In [14]:
k2[56][3573]

0.17047104

In [15]:
k2[1][175]

0.53858167

In [16]:
k2[47][7691]

0.074852765

In [17]:
k2[34][2374]

0.07137603

In [18]:
k2[6153][5483]

0.5114323

In [110]:
data.head()

,1,2,3,4,5,6,7,8,9,10,...,504,505,506,507,508,509,510,511,512,513
0,0.011119,0.519231,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.164003,0.317308,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.181376,0.163462,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.033356,0.278846,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.039611,0.153846,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [26]:
import pandas as pd
result = pd.read_csv("Similarity_Evaluation.csv")
res_ls = []
for index,row in result.iterrows():
    res_ls.append( k2[ int(row['CaseId1']) ][ int(row['CaseId2']) ] )
result.drop( columns=['Similarity'] )
result['Similarity'] = res_ls
result.to_csv("Similarity_Evaluation_.csv")
result

,CaseId1,CaseId2,Similarity
0,1008,1304,0.610564
1,1008,2740,0.475697
2,1008,5788,0.623889
3,1008,8318,0.110415
4,1049,234,0.567991
5,1049,3905,0.728751
6,1049,4951,0.329062
7,1049,8813,0.096881
8,1056,3270,0.175441
9,1056,3573,0.605849
